# Calculate data for REZ regions

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:05:00'
cores = 24
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.6.55.48:8787/status,
Dashboard: http://10.6.55.48:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.55.48:40609,Workers: 0
Dashboard: http://10.6.55.48:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
# client.close()
# cluster.close()

In [6]:
# %load_ext autoreload
# %autoreload 2

In [7]:
import xarray as xr
import numpy as np

In [8]:
import functions as fn

# Load REZ mask

In [9]:
rez_mask = xr.open_dataset(
    '/g/data/w42/dr6273/work/projects/Aus_energy/data/rez_mask_era5_grid.nc'
).REZ

# Load gridded data

In [10]:
years = range(1959, 2021)

Mean solar shortwave radiation downwards

In [11]:
mssrd = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/msdwswrf/msdwswrf_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_REZ_region.zarr',
    consolidated=True
)

100m wind speed

In [12]:
w100 = xr.open_zarr(
    '/g/data/w42/dr6273/work/data/era5/100w/100w_era5_reanalysis_sfc_'+str(years[0])+'-'+str(years[-1])+'_REZ_region.zarr',
    consolidated=True
)

# Calculate regional averages

In [13]:
def REZ_spatial_mean(da, region):
    region_mask = rez_mask.sel(region=region)
    return da.where(region_mask, drop=True).mean(['lat', 'lon'], skipna=True)

In [14]:
def calculate_spatial_means(da):
    
    da_list = []
    for region in rez_mask['region'].values:
        region_mean = REZ_spatial_mean(da, region)
        da_list.append(region_mean)
        
    da_concat = xr.concat(
        da_list,
        dim='region'
    )
    return da_concat

In [15]:
rez_100w = calculate_spatial_means(w100['w100']).compute()

In [16]:
rez_mssrd = calculate_spatial_means(mssrd['msdwswrf']).compute()

# How often to regional means exceed operational thresholds?

- For wind, we use 4 m/s (Ridder report)
- For solar, use 170 W/m^2 (Prasad 2021)

In [17]:
thresh_100w = 4
thresh_mssrd = 170

In [22]:
xr.where(rez_100w < thresh_100w, 1, 0).sum('time') / len(rez_100w.time.values)

<xarray.DataArray 'w100' (region: 39)>
array([0.33286784, 0.41867769, 0.28038616, 0.15337698, 0.21086137,
       0.30686987, 0.55170707, 0.49597795, 0.15136595, 0.17734184,
       0.16988651, 0.11188326, 0.30160036, 0.24233676, 0.4492828 ,
       0.27732271, 0.17881745, 0.31722122, 0.30253687, 0.26369447,
       0.34355957, 0.1711119 , 0.22992655, 0.19899026, 0.15329602,
       0.2706751 , 0.22067547, 0.17106406, 0.15201176, 0.15801172,
       0.21116496, 0.21780152, 0.32633246, 0.6162365 , 0.19508596,
       0.18204466, 0.17609806, 0.34407474, 0.26711855])
Coordinates:
  * region       (region) <U2 'N1' 'N2' 'N3' 'N4' 'N5' ... 'V3' 'V4' 'V5' 'V6'
    region_code  (region) int64 3 7 3 3 2 2 4 4 1 1 1 ... 3 3 3 7 7 6 3 1 1 3 2

In [26]:
xr.where(rez_mask.sel(region='N1'), 0.3, np.nan)

<xarray.DataArray 'REZ' (lat: 721, lon: 1440)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat          (lat) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon          (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
    region       <U2 'N1'
    region_code  int64 ...

Exception ignored in atexit callback: <function close_clusters at 0x14da5d59f130>
Traceback (most recent call last):
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/deploy/spec.py", line 695, in close_clusters
    cluster.close(timeout=10)
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/deploy/cluster.py", line 217, in close
    return self.sync(self._close, callback_timeout=timeout)
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/utils.py", line 339, in sync
    return sync(
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/utils.py", line 406, in sync
    raise exc.with_traceback(tb)
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/utils.py", line 379, in f
    result = yield future
  File "/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/tornado/gen.py"

In [23]:
xr.where(rez_mssrd < thresh_mssrd, 1, 0).sum('time') / len(rez_mssrd.time.values)

<xarray.DataArray 'msdwswrf' (region: 39)>
array([0.61246757, 0.62001141, 0.62078971, 0.61455225, 0.62304918,
       0.63088189, 0.63437046, 0.63772839, 0.62680086, 0.63049918,
       0.63790134, 0.64843327, 0.60749968, 0.60151429, 0.61471416,
       0.61001674, 0.60078566, 0.61274172, 0.62036836, 0.61489816,
       0.61146295, 0.63370439, 0.62587904, 0.61821009, 0.6230455 ,
       0.61942446, 0.60742976, 0.60777567, 0.62171521, 0.62019908,
       0.64573221, 0.66906659, 0.66103884, 0.64725386, 0.62555521,
       0.63739351, 0.64793096, 0.6436181 , 0.63677713])
Coordinates:
  * region       (region) <U2 'N1' 'N2' 'N3' 'N4' 'N5' ... 'V3' 'V4' 'V5' 'V6'
    region_code  (region) int64 3 7 3 3 2 2 4 4 1 1 1 ... 3 3 3 7 7 6 3 1 1 3 2